<a href="https://colab.research.google.com/github/Kgs-Mathaba/Apache_Spark/blob/main/Corona_Virus_EDA_with_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [3]:
!pip install -U pyarrow

     |████████████████████████████████| 23.6 MB 120 kB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 3.0.0
    Uninstalling pyarrow-3.0.0:
      Successfully uninstalled pyarrow-3.0.0


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [5]:
import findspark
findspark.init() # search for spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [10]:
import sys, tempfile

In [11]:
import urllib.request as urllib

In [7]:
BASE_DIR = '/tmp'
CORONA_DATA_FILE = os.path.join(BASE_DIR, 'corona_data.csv')

In [12]:
corona_data = urllib.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv', CORONA_DATA_FILE)

In [14]:
BASE_DIR = '/tmp'
TWITTER_DATA_FILE = os.path.join(BASE_DIR, 'twitter_data.csv')

In [15]:
tweet_data = urllib.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv', TWITTER_DATA_FILE)

In [16]:
!ls /tmp

blockmgr-9dc3ddcc-2b34-4d90-8012-52a3eb5a6800
corona_data.csv
dap_multiplexer.21954c7a4cee.root.log.INFO.20211017-141840.57
dap_multiplexer.INFO
debugger_21ufkzngy6
hsperfdata_root
initgoogle_syslog_dir.0
spark-02a69ef2-2ae9-40c3-ba16-c2725d50541b
spark-c96b0728-fdd3-40f1-9682-b02b32142a4d
twitter_data.csv


In [17]:
corona_df = spark.read.option("inferSchema", "true").csv("/tmp/corona_data.csv", header=True)

In [20]:
corona_df.show()

+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22 00:00:00|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22 00:00:00|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22 00:00:00|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22 00:00:00|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|   

In [21]:
corona_df.count()

28143

In [23]:
twitter_df = spark.read.option("inferSchema", "true").csv("/tmp/twitter_data.csv", header=True)

In [35]:
twitter_df.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

In [36]:
twitter_df.count()

1000

In [37]:
twitter_df.filter("country='USA'")

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [38]:
twitter_df.filter("country='USA'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [39]:
twitter_df.filter("country='USA' and location like '%New%'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen| 

In [40]:
tw_filter_df = twitter_df.filter("country='USA'")

In [41]:
tw_filter_df.explain() # explains transformation

== Physical Plan ==
*(1) Project [_c0#114, geo#115, text#116, user#117, location#118, entities#119, sentiment#120, country#121]
+- *(1) Filter (isnotnull(country#121) && (country#121 = USA))
   +- *(1) FileScan csv [_c0#114,geo#115,text#116,user#117,location#118,entities#119,sentiment#120,country#121] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,USA)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...


In [44]:
tw_filter_df2 = twitter_df.filter("country='USA' and location like '%New%'")

In [45]:
tw_filter_df2.explain()

== Physical Plan ==
*(1) Project [_c0#114, geo#115, text#116, user#117, location#118, entities#119, sentiment#120, country#121]
+- *(1) Filter (((isnotnull(country#121) && isnotnull(location#118)) && (country#121 = USA)) && Contains(location#118, New))
   +- *(1) FileScan csv [_c0#114,geo#115,text#116,user#117,location#118,entities#119,sentiment#120,country#121] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), IsNotNull(location), EqualTo(country,USA), StringContains(location,New)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...


In [46]:
tw_filter_df2.show(5)

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
+---+----+--------------------+---------------+-

In [50]:
from pyspark.sql.functions import col
tw_filter_df = twitter_df.filter(col("location").startswith("N"))

In [51]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#114, geo#115, text#116, user#117, location#118, entities#119, sentiment#120, country#121]
+- *(1) Filter (isnotnull(location#118) && StartsWith(location#118, N))
   +- *(1) FileScan csv [_c0#114,geo#115,text#116,user#117,location#118,entities#119,sentiment#120,country#121] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(location), StringStartsWith(location,N)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...


In [52]:
twitter_df.first()

Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria')

In [54]:
twitter_df.take(5)

[Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria'),
 Row(_c0=1, geo=None, text='"BREAKING: ""this is disappointing', user=' but i took the test"". Arsenal\'s Mikel Arteta tests positive for #coronavirus', location='  -… "', entities='JerryfranksonJF', sentiment='Abuja, Nigeria', country='"[(""Arsenal\'s Mikel Arteta""'),
 Row(_c0=2, geo=None, text=' #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…', user='cek422', location='Pennsylvania, USA', entities='[]', sentiment="{'neg': 0.173, 'neu': 0.71, 'pos': 0.117, 'compound': -0.3767}", country='USA'),
 Row(_c0=3, geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz', locatio

In [56]:
 x = spark.sparkContext.parallelize([1,4,8])
 y = x.flatMap(lambda x: (x, x*x))
 print(x.collect())
 print(y.collect())

[1, 4, 8]
[1, 1, 4, 16, 8, 64]


In [57]:
 x = spark.sparkContext.parallelize([1,4,8])
 y = x.map(lambda x: (x, x*x))
 print(x.collect())
 print(y.collect())

[1, 4, 8]
[(1, 1), (4, 16), (8, 64)]


In [58]:
x

ParallelCollectionRDD[98] at parallelize at PythonRDD.scala:195

In [59]:
twitter_df.select("text").show()

+--------------------+
|                text|
+--------------------+
|What is God sayin...|
|"BREAKING: ""this...|
| #Coronavirus tes...|
| Get ready for ma...|
| The #coronavirus...|
| COVID-19 update ...|
| It’s painful to ...|
| 📽️Friends, I wi...|
| Questions about ...|
|How they’re deali...|
| BREAKING: Democr...|
| “If we close dow...|
| I pity the poor ...|
| We’re the heck w...|
| I don't think sh...|
| Well written, so...|
| 1/2 CDC Director...|
| In all seriousne...|
| Wash your hands....|
| #CoronaVirusCana...|
+--------------------+
only showing top 20 rows



In [60]:
twitter_df.select("text", "user").show()

+--------------------+--------------------+
|                text|                user|
+--------------------+--------------------+
|What is God sayin...|          petodinice|
|"BREAKING: ""this...| but i took the t...|
| #Coronavirus tes...|              cek422|
| Get ready for ma...|        InfectiousDz|
| The #coronavirus...|          vic_gibson|
| COVID-19 update ...|      StewartNgilana|
| It’s painful to ...|           BWheatnyc|
| 📽️Friends, I wi...|             LorseaR|
| Questions about ...|         straightj23|
|How they’re deali...|       _______coolio|
| BREAKING: Democr...|      champagneaylin|
| “If we close dow...|       YorkLawLondon|
| I pity the poor ...|      BeesonMargaret|
| We’re the heck w...|      harrytiffanyiv|
| I don't think sh...|         grammyheath|
| Well written, so...|      barbara_ellena|
| 1/2 CDC Director...|               fatal|
| In all seriousne...|          pwjkmiller|
| Wash your hands....|        Mrrandy123RP|
| #CoronaVirusCana...|           

In [62]:
twitter_df.rdd.map(lambda line: line.text.split(" ")).take(5) # similar to tokenizing

[['What', 'is', 'God', 'saying', 'to', 'us', 'about', '#coronavirus', '?'],
 ['"BREAKING:', '""this', 'is', 'disappointing'],
 ['',
  '#Coronavirus',
  'testing',
  'must',
  'be',
  'made',
  'free',
  'to',
  'the',
  'public',
  'if',
  'we',
  'are',
  'going',
  'to',
  'understand',
  'the',
  'scope',
  'of',
  'this',
  'crisis.',
  'Anything',
  'le…'],
 ['',
  'Get',
  'ready',
  'for',
  'mass',
  'event',
  'crowd',
  'cancellations',
  'across',
  'the',
  'World',
  'starting',
  'this',
  'weekend:',
  'cricket',
  'in',
  '#Australia',
  'in',
  'empty',
  'st…'],
 ['',
  'The',
  '#coronavirus',
  'pandemic',
  'is',
  'revealing',
  'just',
  'how',
  'closely',
  'we',
  'are',
  'all',
  'bound',
  'together...[A',
  'thread]',
  '']]

In [65]:
twitter_df.rdd.flatMap(lambda line: line.text.split(" ")).take(100)

['What',
 'is',
 'God',
 'saying',
 'to',
 'us',
 'about',
 '#coronavirus',
 '?',
 '"BREAKING:',
 '""this',
 'is',
 'disappointing',
 '',
 '#Coronavirus',
 'testing',
 'must',
 'be',
 'made',
 'free',
 'to',
 'the',
 'public',
 'if',
 'we',
 'are',
 'going',
 'to',
 'understand',
 'the',
 'scope',
 'of',
 'this',
 'crisis.',
 'Anything',
 'le…',
 '',
 'Get',
 'ready',
 'for',
 'mass',
 'event',
 'crowd',
 'cancellations',
 'across',
 'the',
 'World',
 'starting',
 'this',
 'weekend:',
 'cricket',
 'in',
 '#Australia',
 'in',
 'empty',
 'st…',
 '',
 'The',
 '#coronavirus',
 'pandemic',
 'is',
 'revealing',
 'just',
 'how',
 'closely',
 'we',
 'are',
 'all',
 'bound',
 'together...[A',
 'thread]',
 '',
 '',
 'COVID-19',
 'update',
 'as',
 'of',
 'this',
 'morning:1.',
 'Death',
 'toll',
 'in',
 'Italy',
 'passes',
 '1,0002.',
 "Arsenal's",
 'head',
 'coach',
 'Arteta',
 'tests',
 'positive3.',
 'US…',
 '',
 'It’s',
 'painful',
 'to',
 'say,',
 'but',
 'as',
 'an']

In [66]:
corona_df.show()

+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22 00:00:00|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22 00:00:00|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22 00:00:00|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22 00:00:00|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|   

In [68]:
corona_df.filter("Country='US'").show()

+---+----------------+-------+------------------+---------+-------------------+---------+-----+---------+----------------+----------------+
|_c0|           State|Country|               Lat|     Long|               Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+---+----------------+-------+------------------+---------+-------------------+---------+-----+---------+----------------+----------------+
| 98|      Washington|     US|           47.4009|-121.4905|2020-01-22 00:00:00|        0|    0|        0|      Washington|      Washington|
| 99|        New York|     US|           42.1657| -74.9481|2020-01-22 00:00:00|        0|    0|        0|        New York|        New York|
|100|      California|     US|           36.1162|-119.6816|2020-01-22 00:00:00|        0|    0|        0|      California|      California|
|101|   Massachusetts|     US|           42.2302| -71.5301|2020-01-22 00:00:00|        0|    0|        0|   Massachusetts|   Massachusetts|
|102|Diamond Princes

In [69]:
corona_df.filter("Country='US'").sort(col("Date"), ascending=False).show()

+-----+----------------+-------+------------------+---------+-------------------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|               Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+-------------------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20 00:00:00|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20 00:00:00|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20 00:00:00|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20 00:00:00|     1177|   23|        0|      California|      California|
|27767

In [70]:
corona_df.filter("Country='US'").sortWithinPartitions([col("Date"), col("Confirmed")], ascending = False).show()

+-----+--------------+-------+-------+---------+-------------------+---------+-----+---------+--------------+--------------+
|  _c0|         State|Country|    Lat|     Long|               Date|Confirmed|Death|Recovered| state_cleaned|          City|
+-----+--------------+-------+-------+---------+-------------------+---------+-----+---------+--------------+--------------+
|27765|      New York|     US|42.1657| -74.9481|2020-03-20 00:00:00|     8310|   42|        0|      New York|      New York|
|27764|    Washington|     US|47.4009|-121.4905|2020-03-20 00:00:00|     1524|   83|        0|    Washington|    Washington|
|27766|    California|     US|36.1162|-119.6816|2020-03-20 00:00:00|     1177|   23|        0|    California|    California|
|27773|    New Jersey|     US|40.2989|  -74.521|2020-03-20 00:00:00|      890|   11|        0|    New Jersey|    New Jersey|
|27776|      Illinois|     US|40.3495| -88.9861|2020-03-20 00:00:00|      585|    5|        0|      Illinois|      Illinois|


In [71]:
corona_df.describe().show()

+-------+-----------------+---------+-----------+------------------+------------------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|              Long|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+------------------+------------------+------------------+------------------+-------------+-----------+
|  count|            28143|    19116|      28143|             28143|             28143|             28143|             28143|             28143|        28143|      14573|
|   mean|          14071.0|     null|       null|30.965553459118834|-34.57031257861667|161.88245744945456| 5.494368048893153| 60.17290267562094|         null|       null|
| stddev|8124.328649186959|     null|       null|19.365472826597646| 80.78375872452575| 2519.847217725942|109.29475709869875|1346.6597829124426| 

In [72]:
corona_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Death: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [73]:
corona_df.filter('Confirmed>10000').sort(col('Confirmed')).show()

+-----+------+-------+-------+--------+-------------------+---------+-----+---------+-------------+----+
|  _c0| State|Country|    Lat|    Long|               Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+------+-------+-------+--------+-------------------+---------+-----+---------+-------------+----+
|24005|  null|   Iran|   32.0|    53.0|2020-03-12 00:00:00|    10075|  429|     2959|       Tehran|null|
|22912|  null|  Italy|   43.0|    12.0|2020-03-10 00:00:00|    10149|  631|      724|         Rome|null|
|27346|France| France|46.2276|  2.2137|2020-03-19 00:00:00|    10871|  243|       12|       France|null|
| 5401| Hubei|  China|30.9756|112.2707|2020-02-02 00:00:00|    11177|  350|      295|        Hubei|null|
|24482|  null|   Iran|   32.0|    53.0|2020-03-13 00:00:00|    11364|  514|     2959|       Tehran|null|
|26253|  null|  Spain|   40.0|    -4.0|2020-03-17 00:00:00|    11748|  533|     1028|       Toledo|null|
|26723|  null|Germany|   51.0|     9.0|2020-03-18 00:00

In [78]:
corona_df.filter('Confirmed>10000').approxQuantile('Confirmed',[0.25, 0.5, 0.75, 0.9, 0.95], 0.9)

[10075.0, 10075.0, 10075.0, 10075.0, 67800.0]

In [79]:
corona_df.agg({"Date":"max"}).collect()

[Row(max(Date)=datetime.datetime(2020, 3, 20, 0, 0))]

In [82]:
corona_df.agg({"Date":"max", "confirmed":"max"}).collect() #get max date and max confirmed

[Row(max(confirmed)=67800, max(Date)=datetime.datetime(2020, 3, 20, 0, 0))]

In [84]:
max_date = corona_df.agg({ "Date":"max"})
max_date.show()

+-------------------+
|          max(Date)|
+-------------------+
|2020-03-20 00:00:00|
+-------------------+



In [87]:
import pyspark.sql.functions as F
corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date")).show()

+--------------+--------------------+-------------------+
|       Country|       State_cleaned|          max(Date)|
+--------------+--------------------+-------------------+
|      Cameroon|             Yaounde|2020-03-20 00:00:00|
|        Cyprus|             Nicosia|2020-03-20 00:00:00|
|            US|            Michigan|2020-03-20 00:00:00|
|         China|             Qinghai|2020-03-20 00:00:00|
|      Portugal|              Lisbon|2020-03-20 00:00:00|
|            US|            Colorado|2020-03-20 00:00:00|
|United Kingdom|      Cayman Islands|2020-03-20 00:00:00|
|            US|            Missouri|2020-03-20 00:00:00|
|         China|              Hainan|2020-03-20 00:00:00|
|     Australia|Australian Capita...|2020-03-20 00:00:00|
|            US|                Guam|2020-03-20 00:00:00|
|        France|             Reunion|2020-03-20 00:00:00|
|      Colombia|        Cundinamarca|2020-03-20 00:00:00|
|          Cuba|              Havana|2020-03-20 00:00:00|
|     Mauritiu

In [89]:
corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), on=['Country', 'State_cleaned', 'Date'], how='inner').show()

+--------------------+--------------------+-------------------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|               Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+-------------------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20 00:00:00|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20 00:00:00|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20 00:00:00|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20 00:00:00|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawa

In [91]:
corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), on=['Country', 'State_cleaned', 'Date'], how='inner').sort("Confirmed", ascending=False).show()

+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+
|       Country|   state_cleaned|               Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|
+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+
|         China|           Hubei|2020-03-20 00:00:00|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|      null|
|         Italy|            Rome|2020-03-20 00:00:00|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null|
|         Spain|          Toledo|2020-03-20 00:00:00|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null|
|       Germany|          Berlin|2020-03-20 00:00:00|27677|          null|   51.0|                9.0|    19848|   67|      180|      null|
|          Iran|    

In [93]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

ws = Window().partitionBy("Country", "State_cleaned").orderBy(col("Date").desc())
corona_df.withColumn("row_num", row_number().over(ws)).show()

+-----+-----+--------+------------------+-------+-------------------+---------+-----+---------+-------------+----+-------+
|  _c0|State| Country|               Lat|   Long|               Date|Confirmed|Death|Recovered|state_cleaned|City|row_num|
+-----+-----+--------+------------------+-------+-------------------+---------+-----+---------+-------------+----+-------+
|27745| null|Cameroon|3.8480000000000003|11.5021|2020-03-20 00:00:00|       20|    0|        0|      Yaounde|null|      1|
|27268| null|Cameroon|3.8480000000000003|11.5021|2020-03-19 00:00:00|       13|    0|        0|      Yaounde|null|      2|
|26791| null|Cameroon|3.8480000000000003|11.5021|2020-03-18 00:00:00|       10|    0|        0|      Yaounde|null|      3|
|26314| null|Cameroon|3.8480000000000003|11.5021|2020-03-17 00:00:00|       10|    0|        0|      Yaounde|null|      4|
|25837| null|Cameroon|3.8480000000000003|11.5021|2020-03-16 00:00:00|        4|    0|        0|      Yaounde|null|      5|
|25360| null|Cam

In [94]:
corona_df.withColumn("row_num", row_number().over(ws)).where(col("row_num")==1).show()

+-----+--------------------+--------------+------------------+---------+-------------------+---------+-----+---------+--------------------+--------+-------+
|  _c0|               State|       Country|               Lat|     Long|               Date|Confirmed|Death|Recovered|       state_cleaned|    City|row_num|
+-----+--------------------+--------------+------------------+---------+-------------------+---------+-----+---------+--------------------+--------+-------+
|27745|                null|      Cameroon|3.8480000000000003|  11.5021|2020-03-20 00:00:00|       20|    0|        0|             Yaounde|    null|      1|
|27859|             Qinghai|         China|           35.7452|  95.9956|2020-03-20 00:00:00|       18|    0|       18|             Qinghai|    null|      1|
|27762|                null|        Cyprus|           35.1264|  33.4299|2020-03-20 00:00:00|       67|    0|        0|             Nicosia|    null|      1|
|27812|            Michigan|            US|           43.3

In [95]:
corona_max_df = corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), on=["Country", "State_cleaned", "Date"], how="inner")

In [96]:
corona_max_df.show()

+--------------------+--------------------+-------------------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|               Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+-------------------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20 00:00:00|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20 00:00:00|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20 00:00:00|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20 00:00:00|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawa

In [100]:
corona_df.groupby("Country").pivot("Date").agg(F.sum("Confirmed")).show(15)

+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------

In [101]:
corona_df.filter("Country=='US'").crosstab("State", "Date").show()

+-------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-----------

In [102]:
corona_max_df.groupby("Country").agg({"Confirmed":"sum"}).show()

+-----------+--------------+
|    Country|sum(Confirmed)|
+-----------+--------------+
|       Chad|             1|
|   Paraguay|            13|
|     Russia|           253|
|    Senegal|            38|
|     Sweden|          1639|
| Cabo Verde|             1|
|     Guyana|             7|
|Philippines|           230|
|   Djibouti|             1|
|  Singapore|           385|
|   Malaysia|          1030|
|       Fiji|             1|
|     Turkey|           359|
|       Iraq|           208|
|    Germany|         19848|
|   Cambodia|            51|
|Afghanistan|            24|
|     Jordan|            85|
|   Maldives|            13|
|     Rwanda|            17|
+-----------+--------------+
only showing top 20 rows



In [103]:
corona_max_df.groupby("Country").agg({"Confirmed":"sum", "Recovered":"sum","Death":"sum"}).orderBy("sum(Confirmed)", ascending=False).show()

+--------------+--------------+----------+--------------+
|       Country|sum(Recovered)|sum(Death)|sum(Confirmed)|
+--------------+--------------+----------+--------------+
|         China|         71266|      3253|         81250|
|         Italy|          4440|      4032|         47021|
|         Spain|          1588|      1043|         20410|
|       Germany|           180|        67|         19848|
|          Iran|          6745|      1433|         19644|
|            US|             0|       244|         19100|
|        France|            12|       450|         12726|
|  Korea, South|          1540|        94|          8652|
|   Switzerland|            15|        54|          5294|
|United Kingdom|            67|       178|          4014|
|   Netherlands|             2|       107|          3003|
|       Austria|             9|         6|          2388|
|       Belgium|             1|        37|          2257|
|        Norway|             1|         7|          1914|
|        Swede

In [104]:
corona_max_df.withColumn("Active", corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death).sort("Active", ascending = False).show()

+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|       Country|   state_cleaned|               Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|Active|
+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|         Italy|            Rome|2020-03-20 00:00:00|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null| 38549|
|       Germany|          Berlin|2020-03-20 00:00:00|27677|          null|   51.0|                9.0|    19848|   67|      180|      null| 19601|
|         Spain|          Toledo|2020-03-20 00:00:00|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null| 17779|
|        France|          France|2020-03-20 00:00:00|27823|        France|46.2276|             2.2137|    12612|  450|

In [105]:
corona_max_df = corona_max_df.withColumn("Active", corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death)

In [106]:
corona_max_df.show(10)

+---------+----------------+-------------------+-----+----------------+--------+---------+---------+-----+---------+----+------+
|  Country|   state_cleaned|               Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|Active|
+---------+----------------+-------------------+-----+----------------+--------+---------+---------+-----+---------+----+------+
| Thailand|         Bangkok|2020-03-20 00:00:00|27666|            null|    15.0|    101.0|      322|    1|       42|null|   279|
|    Japan|         Hiraide|2020-03-20 00:00:00|27667|            null|    36.0|    138.0|      963|   33|      191|null|   739|
|Singapore|       Singapore|2020-03-20 00:00:00|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|   261|
|    Nepal|       Kathmandu|2020-03-20 00:00:00|27669|            null| 28.1667|    84.25|        1|    0|        1|null|     0|
| Malaysia|         Sarawak|2020-03-20 00:00:00|27670|            null|     2.5|    112.5|     10

In [108]:
corona_max_df.groupBy("Country").sum("Active").orderBy("sum(Active)",ascending = False).show()

+--------------+-----------+
|       Country|sum(Active)|
+--------------+-----------+
|         Italy|      38549|
|       Germany|      19601|
|            US|      18856|
|         Spain|      17779|
|        France|      12264|
|          Iran|      11466|
|  Korea, South|       7018|
|         China|       6731|
|   Switzerland|       5225|
|United Kingdom|       3769|
|   Netherlands|       2894|
|       Austria|       2373|
|       Belgium|       2219|
|        Norway|       1906|
|        Sweden|       1607|
|       Denmark|       1327|
|      Portugal|       1009|
|      Malaysia|        940|
|        Canada|        922|
|       Czechia|        829|
+--------------+-----------+
only showing top 20 rows

